# Проект для "Викишоп"

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.  

Задача - обучить модель классифицировать комментарии на позитивные и негативные. Для обучения предоставлен набор данных с разметкой о токсичности правок. Метрика качества модели F1 должна быть не меньше 0.75. 

### Содержание

* [1. Подготовка данных](#chapter1)
* [2. Обучение моделей](#chapter2)

## 1. Подготовка данных <a class="anchor" id="chapter1"></a>

In [1]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [4]:
data = pd.read_csv('toxic_comments.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [29]:
data['toxic'].sum()

490

In [6]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Столбец *Unnamed: 0* дублирует номера строк, удалим его.

In [7]:
data = data.drop(['Unnamed: 0'], axis=1)

Данные загружены.  
Подготовим их.

In [8]:
data = data.sample(5000).reset_index(drop=True)

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
tokenizer.model_max_length = 64
tokenized = data['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [10]:
attention_mask.shape

(5000, 64)

In [11]:
#config = transformers.BertConfig.from_json_file('bert_config.json')
model = transformers.BertModel.from_pretrained("bert-base-multilingual-uncased")

In [12]:
%%time
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/50 [00:00<?, ?it/s]

Wall time: 5min 49s


In [13]:
features = np.concatenate(embeddings)

In [14]:
features

array([[-0.09723997, -0.10773207, -0.02637959, ...,  0.07632776,
        -0.10432892, -0.09856485],
       [-0.08844389,  0.05526489,  0.08140454, ..., -0.00573247,
         0.03548515, -0.01331934],
       [-0.1881977 , -0.05875487,  0.00701007, ..., -0.04446445,
         0.05300491, -0.03400236],
       ...,
       [-0.14999884, -0.07863538,  0.10773139, ...,  0.1259378 ,
        -0.02502017, -0.08655902],
       [-0.19202243, -0.04807636, -0.04411079, ..., -0.02882728,
         0.05329894, -0.07731294],
       [-0.12386946,  0.02875497, -0.01680586, ...,  0.02542338,
        -0.0076592 ,  0.10311651]], dtype=float32)

## 2. Обучение моделей<a class="anchor" id="chapter2"></a>

In [15]:
features = pd.DataFrame(features)
features_train, features_test = train_test_split(features, test_size=0.2)
target_train, target_test = train_test_split(data['toxic'], test_size=0.2)

In [23]:
model = LogisticRegression()
model.fit(features_train, target_train)

C:\Users\home\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [24]:
predicted = model.predict(features_test)

In [25]:
predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [26]:
target_test

2026    0
1389    0
587     0
3181    1
4772    0
       ..
2149    0
2125    0
2226    0
4016    0
4059    0
Name: toxic, Length: 1000, dtype: int64

In [27]:
accuracy_score(predicted, target_test)

0.903

In [28]:
f1_score(target_test, predicted)

0.0